# Environment Setup and Imports

In [ ]:
!pip install -q -U transformers peft datasets accelerate trl bitsandbytes

In [ ]:
from huggingface_hub import notebook_login
import pandas as pd
from datasets import load_dataset
from IPython.display import HTML, display
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline as hf_pipeline, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel

## Dataset Preparation

In [ ]:
# --- 2.1 Load Raw Dataset ---
dataset_name = "b-mc2/sql-create-context"
dataset = load_dataset(dataset_name, split="train")

In [ ]:
def display_table(dataset_or_sample):
  pd.set_option("display.max_colwidth", None)
  pd.set_option("display.width", None)
  pd.set_option("display.max_rows", None)
  if isinstance(dataset_or_sample, dict):
      df = pd.DataFrame(dataset_or_sample, index=[0])
  else:
      df = pd.DataFrame(dataset_or_sample)
  html = df.to_html().replace("\n", "<br>")
  styled_html = f"""<style> .dataframe th, .dataframe tbody td {{ text-align: left; padding-right: 30px; }} </style> {html}"""
  display(HTML(styled_html))

print("Displaying a few samples from the raw dataset:")
display_table(dataset.select(range(3)))

In [ ]:
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

print(f"Training dataset contains {len(train_dataset)} text-to-SQL pairs")
print(f"Test dataset contains {len(test_dataset)} text-to-SQL pairs")


In [ ]:
PROMPT_TEMPLATE = """You are a powerful text-to-SQL model. Given the SQL tables and natural language question, your job is to write SQL query that answers the question.

### Table:
{context}

### Question:
{question}

### Response:
{output}"""

def apply_prompt_template(row):
  prompt = PROMPT_TEMPLATE.format(
      question=row["question"],
      context=row["context"],
      output=row["answer"], # During training, the 'answer' is the target output
  )
  return {"prompt": prompt}

train_dataset_formatted = train_dataset.map(apply_prompt_template)
print("\nDisplaying a sample from the training dataset after applying prompt template:")
display_table(train_dataset_formatted.select(range(1)))

In [ ]:
base_model_id = "HuggingFaceTB/SmolLM2-135M"
MAX_LENGTH = 256 # You can adjust this

tokenizer = AutoTokenizer.from_pretrained(
  base_model_id,
  model_max_length=MAX_LENGTH,
  padding_side="left", # Important for decoder-only models
  add_eos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token # Set pad token to EOS token

def tokenize_and_pad_to_fixed_length(sample):
  result = tokenizer(
      sample["prompt"],
      truncation=True,
      max_length=MAX_LENGTH,
      padding="max_length", # Pad to MAX_LENGTH
  )
  result["labels"] = result["input_ids"].copy() # For Causal LM, labels are usually input_ids shifted
  return result

tokenized_train_dataset = train_dataset_formatted.map(tokenize_and_pad_to_fixed_length, batched=True) # batched=True can speed this up
# tokenized_test_dataset = test_dataset.map(apply_prompt_template).map(tokenize_and_pad_to_fixed_length, batched=True) # Also tokenize test for eval

assert all(len(x["input_ids"]) == MAX_LENGTH for x in tokenized_train_dataset)

print("\nDisplaying a tokenized sample from the training dataset:")
display_table(tokenized_train_dataset.select(range(1)))

## Model Initialization

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\nUsing device: {device}")

quantization_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=quantization_config,
    # device_map="auto" # Automatically distribute model on available GPUs if any
)
print("\nBase model loaded with 4-bit quantization.")


## Pre-Finetuning Inference

In [ ]:
pipeline_tokenizer = AutoTokenizer.from_pretrained(base_model_id)
if pipeline_tokenizer.pad_token is None:
    pipeline_tokenizer.pad_token = pipeline_tokenizer.eos_token

text_gen_pipeline_base = hf_pipeline(
    task="text-generation",
    model=base_model, # Use the quantized base_model
    tokenizer=pipeline_tokenizer,
    device_map="auto"
)

sample_idx_for_test = 1
if len(test_dataset) > sample_idx_for_test:
    sample = test_dataset[sample_idx_for_test]
    prompt_for_base_inference = PROMPT_TEMPLATE.format(
      context=sample["context"], question=sample["question"], output="" # Leave output blank for generation
    )

    print("\n--- Running Inference with BASE (Quantized) Model (Before Fine-tuning) ---")
    with torch.no_grad():
      response = text_gen_pipeline_base(
          prompt_for_base_inference,
          max_new_tokens=100, # Shorter for quick test
          repetition_penalty=1.15,
          return_full_text=False,
          eos_token_id=pipeline_tokenizer.eos_token_id, # Important for stopping
          pad_token_id=pipeline_tokenizer.pad_token_id  # Also good practice
      )

    print("\nDisplaying BASE model inference result:")
    display_table({"prompt": prompt_for_base_inference, "generated_query_base_model": response[0]["generated_text"]})
else:
    print(f"\nSkipping pre-finetuning inference as test_dataset has less than {sample_idx_for_test+1} samples.")

### System: You are a powerful text-to-SQL model. Given the SQL tables and natural language question, your job is to write SQL query that answers the question.

### Question: What is the lowest numbered game against Phoenix with a record of 29-17?

### Context: CREATE TABLE table_name_61 (game INTEGER, opponent VARCHAR, record VARCHAR)

## Partial Fine-Tuning Setup

In [ ]:
# --- 5.1 Prepare model for k-bit training ---
base_model.gradient_checkpointing_enable() # Saves memory during training
peft_ready_model = prepare_model_for_kbit_training(base_model) # Prepares quantized model for PEFT
print("\nModel prepared for k-bit training (QLoRA).")

In [ ]:
# --- 5.2 LoRA Configuration ---
peft_config = LoraConfig(
  task_type="CAUSAL_LM",
  r=32,
  lora_alpha=64,
  lora_dropout=0.1,
  target_modules=[ # Ensure these modules exist in your model (SmolLM2 might have different names)
      "q_proj", "k_proj", "v_proj", "o_proj",
      # "gate_proj", "up_proj", "down_proj", # These might not be in SmolLM-135M
      # "lm_head", # Usually not targeted with LoRA for stability, but can be if needed.
                  # For SmolLM2, let's check its architecture if these are appropriate.
                  # If unsure, start with query, key, value, output projections.
  ],
  bias="none",
)

# --- 5.3 Get PEFT Model ---
peft_model = get_peft_model(peft_ready_model, peft_config)
print("\nPEFT model created with LoRA adapters.")
peft_model.print_trainable_parameters()

## Training

In [ ]:
output_dir = "./sql_smollm2_qlora_finetuned"

training_args = TrainingArguments(
  output_dir=output_dir,
  per_device_train_batch_size=2, # Adjust based on your GPU memory
  gradient_accumulation_steps=4,  # Effective batch size = 2 * 4 = 8
  gradient_checkpointing=True,    # Already enabled on model, but good to have here too
  optim="paged_adamw_8bit",       # Optimizer suitable for QLoRA
  bf16=torch.cuda.is_bf16_supported(), # Use bf16 if available, otherwise fp16 might be an alternative or disable
  fp16=not torch.cuda.is_bf16_supported() and torch.cuda.is_available(), # Fallback to fp16 if no bf16
  learning_rate=2e-5,
  lr_scheduler_type="constant",
  max_steps=500,                  # Number of training steps
  save_strategy="steps",          # Save checkpoints at save_steps interval
  save_steps=100,                 # Save a checkpoint every 100 steps
  logging_steps=10,               # Log training metrics every 10 steps
  warmup_steps=5,
  ddp_find_unused_parameters=False if torch.cuda.device_count() > 1 else None, # Only for DDP
  report_to="none", # Set to "tensorboard", "wandb", etc. if you want logging
  # run_name=f"SmolLM2-SQL-QLoRA-{datetime.now().strftime('%Y-%m-%d-%H-%M-%s')}", # If using wandb/mlflow
)

# Data Collator
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# Trainer
trainer = Trainer(
  model=peft_model,
  train_dataset=tokenized_train_dataset,
  # eval_dataset=tokenized_test_dataset, # You would add this for evaluation during training
  data_collator=data_collator,
  args=training_args,
)

In [ ]:
# Compatibility with gradient checkpointing
if hasattr(peft_model, 'config') and hasattr(peft_model.config, 'use_cache'):
    peft_model.config.use_cache = False
elif hasattr(base_model, 'config') and hasattr(base_model.config, 'use_cache'): # if peft_model doesn't have it directly
    base_model.config.use_cache = False

print("\nTrainer initialized. Starting training...")

In [ ]:
train_result = trainer.train()
print("\nTraining finished.")

In [ ]:
# --- Save training metrics ---
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)

# --- Save the fine-tuned PEFT model (adapters) ---
# The trainer automatically saves the model to output_dir/checkpoint-xxx and the final model
# But we can explicitly save the final adapter model as well
final_adapter_dir = f"/kaggle/working/idlyvadda"
peft_model.save_pretrained(final_adapter_dir)
tokenizer.save_pretrained(final_adapter_dir) # Save tokenizer with adapter for convenience
print(f"Fine-tuned PEFT adapters saved to: {final_adapter_dir}")

## Post Fine-Tuning Inference

In [ ]:
# ==============================================================================
# SECTION 8: INTERACTIVE POST-FINETUNING INFERENCE (Demonstration)
# ==============================================================================
# This section focuses on loading your fine-tuned model and allowing you to
# interactively provide context and questions to get SQL queries.

print("\n--- SECTION 8: Interactive Post-Finetuning Inference ---")

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline as hf_pipeline
from peft import PeftModel
import torch

# --- Configuration (Should match training/saving) ---
base_model_id = "HuggingFaceTB/SmolLM2-135M"
# This output_dir should be where your TrainingArguments saved the final model/checkpoints
# And where 'final_adapter' subdirectory was created.
# Example: output_dir_from_training = "./sql_smollm2_qlora_finetuned"
output_dir_from_training = training_args.output_dir # Use the one defined in TrainingArguments
final_adapter_dir = f"{output_dir_from_training}/final_adapter" # This is where we saved it

# Define the prompt template again (must be identical to the one used for training)
PROMPT_TEMPLATE = """You are a powerful text-to-SQL model. Given the SQL tables and natural language question, your job is to write SQL query that answers the question.

### Table:
{context}

### Question:
{question}

### Response:
{output}"""

# --- 8.1 Load Base Quantized Model ---
print(f"\nLoading base model '{base_model_id}' with quantization...")
quantization_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16,
)

# Ensure the device is set correctly
device = 'cuda' if torch.cuda.is_available() else 'cpu'

loaded_base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=quantization_config,
    device_map="auto" # Automatically map to device
)
print("Base model for inference loaded.")

# --- 8.2 Load PEFT Adapters ---
print(f"Loading PEFT adapters from: {final_adapter_dir}")
# Ensure the base model is on the correct device before loading adapters if not using device_map="auto"
# loaded_base_model.to(device)
finetuned_model = PeftModel.from_pretrained(loaded_base_model, final_adapter_dir)
finetuned_model.eval() # Set to evaluation mode
print("PEFT adapters loaded onto the base model.")

# --- 8.3 Load Tokenizer (the one saved with adapters is best) ---
print(f"Loading tokenizer from: {final_adapter_dir}")
inference_tokenizer = AutoTokenizer.from_pretrained(final_adapter_dir)
if inference_tokenizer.pad_token is None: # Ensure pad token is set
    inference_tokenizer.pad_token = inference_tokenizer.eos_token
print("Tokenizer loaded.")

# --- 8.4 Create Inference Pipeline ---
text_gen_pipeline_finetuned = hf_pipeline(
    task="text-generation",
    model=finetuned_model,
    tokenizer=inference_tokenizer,
    device_map="auto" # Let pipeline handle device mapping
)
print("Inference pipeline with fine-tuned model created.")

# --- 8.5 Interactive Function to Generate SQL ---
def generate_sql_query(table_context, natural_language_question, max_new_tokens=150):
    """
    Generates an SQL query given table context and a natural language question
    using the fine-tuned model.
    """
    prompt = PROMPT_TEMPLATE.format(
      context=table_context,
      question=natural_language_question,
      output="" # Leave output blank for generation
    )

    print("\n--- Generating SQL Query ---")
    print(f"Table Context:\n{table_context}")
    print(f"Question:\n{natural_language_question}")

    with torch.no_grad(): # Ensure no gradients are computed during inference
      response = text_gen_pipeline_finetuned(
          prompt,
          max_new_tokens=max_new_tokens,
          repetition_penalty=1.15, # You can tune this
          temperature=0.7,        # You can tune this (lower for more deterministic, higher for more creative)
          do_sample=True,         # Recommended for more natural outputs, set False for greedy
          top_k=50,               # Consider only the top_k tokens
          top_p=0.95,             # Nucleus sampling
          return_full_text=False, # We only want the generated part
          eos_token_id=inference_tokenizer.eos_token_id,
          pad_token_id=inference_tokenizer.pad_token_id
      )

    generated_sql = response[0]["generated_text"].strip()
    print(f"\nGenerated SQL Query:\n{generated_sql}")
    return generated_sql

# --- 8.6 DEMONSTRATION: Provide your custom context and question here ---

# Example 1: Simple Query
my_table_context_1 = """
CREATE TABLE employees (
    EmployeeID int,
    FirstName varchar(255),
    LastName varchar(255),
    Department varchar(255),
    Salary int
)
"""
my_question_1 = "Show me the first names of all employees in the 'Sales' department."
generated_sql_1 = generate_sql_query(my_table_context_1, my_question_1)

In [ ]:
# Example 2: A slightly more complex query
my_table_context_2 = """
CREATE TABLE products (
    ProductID int,
    ProductName varchar(255),
    CategoryID int,
    Price decimal
);
CREATE TABLE categories (
    CategoryID int,
    CategoryName varchar(255)
)
"""
my_question_2 = "What are the names of products that belong to the 'Electronics' category and cost more than 500?"
generated_sql_2 = generate_sql_query(my_table_context_2, my_question_2)